In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Essential_Thrombocythemia"
cohort = "GSE159514"

# Input paths
in_trait_dir = "../../input/GEO/Essential_Thrombocythemia"
in_cohort_dir = "../../input/GEO/Essential_Thrombocythemia/GSE159514"

# Output paths
out_data_file = "../../output/preprocess/Essential_Thrombocythemia/GSE159514.csv"
out_gene_data_file = "../../output/preprocess/Essential_Thrombocythemia/gene_data/GSE159514.csv"
out_clinical_data_file = "../../output/preprocess/Essential_Thrombocythemia/clinical_data/GSE159514.csv"
json_path = "../../output/preprocess/Essential_Thrombocythemia/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# Based on background information, this dataset contains gene expression profiling data from microarray
is_gene_available = True

# 2.1 Data Availability
# For trait: The trait is available from the 'disease' field (row 0)
trait_row = 0

# For age and gender: Not available in the sample characteristics dictionary
age_row = None
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value: str) -> int:
    """Convert trait value to binary (0 for control, 1 for case)"""
    if value is None:
        return None
    
    # Extract value after colon if applicable
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Based on the context, this is a study on myelofibrosis
    # Essential Thrombocythemia (ET) specifically relates to PET (Post-ET myelofibrosis)
    if 'PET' in value:  # Post-ET myelofibrosis is related to Essential Thrombocythemia
        return 1
    else:
        return 0  # Other conditions (PPV, Overt-PMF, Pre-PMF) are not Essential Thrombocythemia

def convert_age(value: str) -> Optional[float]:
    """Convert age value to continuous number"""
    # Not available in this dataset
    return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender value to binary (0 for female, 1 for male)"""
    # Not available in this dataset
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Conduct initial filtering on usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create a proper DataFrame from the sample characteristics dictionary
    # The dictionary has two columns (0 and 1) which need to be converted to a DataFrame with proper shape
    
    # First, create a dictionary where keys are column names and values are column data
    sample_chars = {
        0: ['disease: PPV', 'disease: Overt-PMF', 'disease: PET', 'disease: Pre-PMF'],
        1: ['driver mutation: JAK2V617F', 'driver mutation: CALR Type 1', 
            'driver mutation: MPL', 'driver mutation: TN', 
            'driver mutation: CALR Type 2', 'driver mutation: CALR', 
            'driver mutation: JAK2 ex12']
    }
    
    # For the geo_select_clinical_features function, we need a DataFrame where each row is a feature
    # and each column is a sample. For this simple example, reshape it appropriately
    clinical_data = pd.DataFrame()
    for i, values in sample_chars.items():
        # Add each feature as a row
        row_df = pd.DataFrame([values])
        clinical_data = pd.concat([clinical_data, row_df], ignore_index=True)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical data
    print("Preview of selected clinical data:")
    preview = preview_df(selected_clinical_df)
    print(preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save clinical data as CSV
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import os
import json
import pandas as pd
from typing import Callable, Optional, Dict, Any

# Check for required data files
clinical_data_path = os.path.join(in_cohort_dir, "sample_characteristics.csv")
if os.path.exists(clinical_data_path):
    clinical_data = pd.read_csv(clinical_data_path)
    print(f"Clinical data loaded with shape {clinical_data.shape}")
else:
    print(f"Clinical data file not found at {clinical_data_path}")
    clinical_data = pd.DataFrame()

metadata_path = os.path.join(in_cohort_dir, "metadata.txt")
if os.path.exists(metadata_path):
    with open(metadata_path, 'r') as f:
        metadata = f.read()
        print(f"Metadata file loaded ({len(metadata)} characters)")
else:
    metadata = ""
    print(f"Metadata file not found at {metadata_path}")

# Check for gene expression data
matrix_path = os.path.join(in_cohort_dir, "matrix.csv")
is_gene_available = os.path.exists(matrix_path)
if is_gene_available:
    print(f"Gene expression matrix file found at {matrix_path}")
else:
    print("Gene expression matrix file not found, setting is_gene_available to False")

# Since we don't have clinical data, we can't identify trait, age, and gender rows
# Set all to None to indicate data is not available
trait_row = None
age_row = None
gender_row = None

# Define conversion functions for completeness, but they won't be used since data is not available
def convert_trait(value):
    """Convert trait value to binary (0 for control, 1 for Essential Thrombocythemia)"""
    if pd.isna(value):
        return None
    value = str(value).lower()
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if any(term in value.lower() for term in ['et', 'essential thrombocythemia', 'thrombocythaemia']):
        return 1
    elif any(term in value.lower() for term in ['control', 'healthy', 'normal']):
        return 0
    return None

def convert_age(value):
    """Convert age value to continuous numeric format"""
    if pd.isna(value):
        return None
    value = str(value)
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    import re
    age_match = re.search(r'(\d+)', value)
    if age_match:
        return float(age_match.group(1))
    return None

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)"""
    if pd.isna(value):
        return None
    value = str(value).lower()
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if any(term in value for term in ['female', 'f', 'woman']):
        return 0
    elif any(term in value for term in ['male', 'm', 'man']):
        return 1
    return None

# Check if trait data is available
is_trait_available = trait_row is not None
print(f"Trait data available: {is_trait_available}")

# Save metadata using the library function
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

print(f"Cohort metadata saved to {json_path}")
print(f"Dataset analysis complete. Gene data available: {is_gene_available}, Trait data available: {is_trait_available}")

# We skip clinical feature extraction since trait_row is None (data not available)
if trait_row is not None:
    # Use geo_select_clinical_features function to extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the data
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical data:")
    print(preview)
    
    # Save clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=True)
    print(f"Clinical data saved to {out_clinical_data_file}")
else:
    print("Skipping clinical feature extraction as trait data is not available")


### Step 4: Gene Data Extraction

In [ ]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


### Step 5: Gene Identifier Review

In [ ]:
# Looking at the gene identifiers, I can see they follow the format like "11715100_at", "11715101_s_at", etc.
# These are not human gene symbols but appear to be Affymetrix probe IDs
# They will require mapping to human gene symbols for meaningful biological interpretation

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Identify the relevant columns for mapping
probe_id_column = 'ID'
gene_symbol_column = 'Gene Symbol'

# 2. Get a gene mapping dataframe by extracting the relevant columns
mapping_df = get_gene_mapping(gene_annotation, probe_id_column, gene_symbol_column)
print(f"Created mapping between {probe_id_column} and {gene_symbol_column}")
print(f"Mapping dataframe shape: {mapping_df.shape}")
print("First 5 rows of mapping dataframe:")
print(mapping_df.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Mapped gene expression data shape: {gene_data.shape}")
print("First 10 genes in mapped expression data:")
print(gene_data.index[:10])

# 4. Normalize gene symbols (if needed)
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene expression data shape: {gene_data.shape}")
print("First 10 genes after normalization:")
print(gene_data.index[:10])

# 5. Save the processed gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Re-acquire gene data as it seems to be missing in this context
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_data = get_genetic_data(matrix_file)

# Extract gene annotation and mapping again
gene_annotation = get_gene_annotation(soft_file)
mapping_df = get_gene_mapping(gene_annotation, 'ID', 'Gene Symbol')
gene_data = apply_gene_mapping(gene_data, mapping_df)

# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Define trait availability based on previous analysis steps
trait_row = None  # From previous steps we determined trait data wasn't properly available
is_gene_available = True
is_trait_available = False

# 2. Check if trait data is available before attempting to link
if trait_row is not None:
    print("Loading the original clinical data...")
    background_info, clinical_data = get_background_and_clinical_data(matrix_file)

    print("Extracting clinical features...")
    # Define conversion functions
    def convert_trait(value):
        if isinstance(value, str) and ':' in value:
            value = value.split(':', 1)[1].strip()
        return 1 if 'PET' in str(value) else 0
        
    def convert_age(value):
        return None
        
    def convert_gender(value):
        return None
    
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )

    print("Clinical data preview:")
    print(preview_df(selected_clinical_df))

    # Save the clinical data to a CSV file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")

    # Link clinical and genetic data using the normalized gene data
    print("Linking clinical and genetic data...")
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape: {linked_data.shape}")

    # 3. Handle missing values in the linked data
    print("Handling missing values...")
    linked_data = handle_missing_values(linked_data, trait)
    print(f"Linked data shape after handling missing values: {linked_data.shape}")

    # 4. Check if trait is biased
    print("Checking for bias in trait distribution...")
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
else:
    print("Trait data is not available, skipping clinical data extraction and linking.")
    # Set variables for final validation
    is_biased = None  # Cannot determine bias without trait data
    linked_data = pd.DataFrame()  # Empty dataframe for validation

# 5. Final validation
note = "Dataset contains gene expression data from patients with myelofibrosis, but Essential Thrombocythemia specific annotations are not available in proper format."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

print(f"Dataset usability: {is_usable}")

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies due to lack of proper trait annotations.")


### Step 9: Data Normalization and Linking

In [ ]:
# 1. Reload gene data and perform gene mapping since it's not available in this context
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_data = get_genetic_data(matrix_file)
gene_annotation = get_gene_annotation(soft_file)
mapping_df = get_gene_mapping(gene_annotation, 'ID', 'Gene Symbol')
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Normalize gene symbols in the gene expression data
print("Normalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Define the trait availability based on previous steps
is_trait_available = trait_row is not None
is_gene_available = True  # We have gene data

# Skip clinical data extraction and linking since trait data is unavailable
print("Trait data is not available, skipping clinical data extraction and linking.")
linked_data = pd.DataFrame()  # Empty dataframe since we can't link
is_biased = False  # Cannot determine bias for non-existent trait data

# Final validation
note = "Dataset contains gene expression data from myelofibrosis patients, but Essential Thrombocythemia specific annotations are not properly available for trait-gene association studies."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

print(f"Dataset usability: {is_usable}")

# Save linked data if usable (will not execute since is_usable will be False)
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies due to lack of proper trait annotations.")